<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Aug 17th, 2021</h5>

### Step 0: Import Python packages and initiate koster database

In [6]:
# Import required modules
import getpass
import utils.t12_utils as t12

# Add Zooniverse credentials
zoo_user = getpass.getpass('Enter your Zooniverse user')
zoo_pass = getpass.getpass('Enter your Zooniverse password')

# TODO replace the code below to initiate koster db for one line (code below is from "#3.Build and populate koster database")
import os
if os.path.exists("koster_lab.db"):
  os.remove("koster_lab.db")
else:
  print("There are no previous database versions")

# Initiate the db
%run -i "../db_starter/init.py"

# Populate the db with info from the csv files
%run -i "../db_starter/static.py"

# Populate the db with info of subjects uploaded to Zooniverse
%run -i "../db_starter/subjects_uploaded.py" --user $zoo_user --passw $zoo_pass

Enter your Zooniverse user········
Enter your Zooniverse password········
There are no previous database versions


FileNotFoundError: [Errno 2] No such file or directory: '../db_starter/db_csv_info/sites_koster.csv'

C:\Users\Victor\koster_data_management\db_starter\subjects_uploaded.py:111: FutureWarning: The default value of regex will change from True to False in a future version.
  clip_filenames = meta_df["filename"].str.replace(".mp4", "")


There are clip subjects that don't have movie_id. The movie filenames are {'01484002_1.mov', '990810 TMBL-ROV 1999 Väderöarna-Storö Tape 46.mov', '041119 TMBL-ROV 2004 Yttre Vattenholmen.mov', '000203 TMBL-ROV 2000 säcken Tape 56.mov', '040521 TMBL-ROV 2004 Säcken NO.mov', '031023 TMBL-ROV 2003 Säcken Bioerosion.mov', '040415 TMBL-ROV 2004 Valkadaver del 1.mov', '020629 TMBL-ROV 2002 O Väderöarna.mov', '990506 TMBL-ROV 1999 Revet Säcken Tape 42_SELECTWS.mov', '01484001_1.mov', '01724003.mov', '01484003_1.mov', '01725006_1.mov', '970708 TMBL-ROV 1997 Tape 2 Krugglöbranten.mov', '970708 TMBL-ROV 1997 Tape 2 S Kostergrund.mov', '010424 Säckenrevet beta Tape 74.mov', '000203 TMBL-ROV 2000 Säckenrevet Tape 56.mov', '990806 TMBL-ROV 1999 Yttre Vattenholmen Tape 45.mov', '990622 TMBL-ROV 1999 V Yttre Vattenholmen Tape 44.mov', '01727001_1.mov', '040220 TMBL-ROV 2004 Säckenrevet alfa.mov', '01484004_1.mov', '000203 TMBL-ROV 2000 Säcken revet EJ numrerade band.mov', '990304 TMBL-R

RuntimeError: No active exception to reraise

### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

In [11]:
# TODO Display a selectable list of workflow ids with their names and a list of versions of the workflow of interest
#workflow_id, workflow_version, subj_type = t13.get_workflows(zoo_user, zoo_pass)
    
# Temporary example objects
workflow_id = 17719
workflow_version = 0.0
subj_type = "clip"

# Retrieve classifications from the workflow of interest
total_df, class_df = t12.get_classifications(workflow_id, workflow_version, subj_type, zoo_user, zoo_pass)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 3


### Step 2: Aggregate classifications received on the workflow of interest

In [12]:
# TODO Display a selectable list of min number of users, agreement, 
#agg_user, min_users = t13.set_aggregation_parameters()

# Specify the agreement threshold required among cit scientists
agg_users = 0.8
min_users = 3

class_df1 = t12.aggregrate_classifications(class_df, subj_type, agg_users, min_users)

NameError: name 'class_df' is not defined

### Step 3: Summarise the number of classifications based on the agreement specified

### Step 4: Use the subject explorer widget to get more information about specific subjects and their aggregated classifications

In [ ]:
# Get all classified subjects from specified workflows
total_df, class_df = t13.get_exports(user, password)

clips_df = t13.process_clips(total_df, class_df, clips_workflow_id, clips_workflow_version)
frames_df = t13.process_frames(db_path, total_df, frames_workflow_id,
                               frames_workflow_version, 
                               "https://drive.google.com/file/d/1z72CqTtEBtqk6936H1YNrCjc5NRopF0g/view?usp=sharing")

t13.launch_viewer(total_df, clips_df, frames_df)

In [ ]:
# END